# Домашнее задание - 5. Декораторы

## Импорт библиотек, установка констант

In [2]:
import requests
import time
import re

from random import randint

In [3]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1
Реализуйте декоратор benchmark(func), выводящий время, которое заняло выполнение декорируемой функции

In [4]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    def wrapper(*args, **kwargs):
        start_time = time.time()  # Capture the start time
        result = func(*args, **kwargs)  # Execute the decorated function and capture its result
        end_time = time.time()  # Capture the end time
        duration = end_time - start_time  # Calculate the duration
        print(f"{func.__name__} took {duration:.4f} seconds to execute.")
        return result  # Return the result of the decorated function
    return wrapper

In [5]:
import requests

@benchmark
def download_and_process_book(url):
    response = requests.get(url)
    # Here, you would process the book content in some way
    print(f"Downloaded {len(response.content)} bytes.")

# Now, calling the function will automatically print the execution time.
download_and_process_book(BOOK_PATH)


Downloaded 1427675 bytes.
download_and_process_book took 2.2719 seconds to execute.


## Задание 2
Реализуйте декоратор logging(func), который выводит параметры с которыми была вызвана функция

In [6]:
def logging(func):
    """
    Декоратор, который выводит параметры, с которыми была вызвана функция.
    """
    def wrapper(*args, **kwargs):
        args_repr = [repr(a) for a in args]  # Создаем список представлений всех позиционных аргументов
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # Создаем список строковых представлений всех именованных аргументов
        signature = ", ".join(args_repr + kwargs_repr)  # Собираем сигнатуру из аргументов
        print(f"Вызов функции {func.__name__} с аргументами ({signature})")  # Выводим информацию о вызове функции
        result = func(*args, **kwargs)  # Вызываем саму функцию
        return result  # Возвращаем результат выполнения функции
    return wrapper

In [7]:
@logging
def download_and_process_book(url):
    response = requests.get(url)
    print(f"Загружено {len(response.content)} байт.")

# Теперь при вызове функции будет выводиться информация о параметрах вызова.
download_and_process_book(BOOK_PATH)

Вызов функции download_and_process_book с аргументами ('https://www.gutenberg.org/files/2638/2638-0.txt')
Загружено 1427675 байт.


## Задание 3
Реализуйте декоратор counter(func), считающий и выводящий количество вызовов декорируемой функции

In [8]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции.
    """
    # Инициализация счетчика вызовов функции
    func.call_count = 0

    def wrapper(*args, **kwargs):
        # Инкрементируем счетчик при каждом вызове функции
        func.call_count += 1
        print(f"Функция {func.__name__} была вызвана {func.call_count} раз(а)")
        # Вызываем саму функцию
        return func(*args, **kwargs)

    return wrapper

In [9]:
@counter
def download_and_process_book(url):
    response = requests.get(url)
    print(f"Загружено {len(response.content)} байт.")

# Вызываем функцию несколько раз, чтобы проверить, как работает счетчик вызовов
download_and_process_book(BOOK_PATH)
download_and_process_book(BOOK_PATH)

Функция download_and_process_book была вызвана 1 раз(а)
Загружено 1427675 байт.
Функция download_and_process_book была вызвана 2 раз(а)
Загружено 1427675 байт.


## Задание 4
Реализуйте декоратор memo(func), запоминающий результаты исполнения функции func, чьи аргументы *args должны быть хешируемыми. Сравните время выполнения рекурсивной реализации расчета чисел Фибоначчи без декоратора и с ним


In [11]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func, чьи аргументы *args должны быть хешируемыми.
    """
    cache = {}  # Словарь для хранения результатов вызова функции

    def wrapper(*args):
        if args in cache:
            return cache[args]  # Возвращаем результат из кэша, если он там есть
        result = func(*args)  # Выполняем функцию и сохраняем результат
        cache[args] = result  # Сохраняем результат выполнения функции в кэше
        return result

    wrapper.cache = cache
    return wrapper

# Рекурсивная функция для вычисления чисел Фибоначчи
def fib(n):
    if n <= 1:
        return n
    else:
        return fib(n-1) + fib(n-2)

# Декорированная версия функции fib
@memo
def fib_memo(n):
    if n <= 1:
        return n
    else:
        return fib_memo(n-1) + fib_memo(n-2)

# Измеряем время выполнения обеих функций
def benchmark_fib(n):
    start_time = time.time()
    print(fib(n))
    print(f"Fibonacci without memoization took {time.time() - start_time:.4f} seconds.")

    start_time = time.time()
    print(fib_memo(n))
    print(f"Fibonacci with memoization took {time.time() - start_time:.4f} seconds.")

benchmark_fib(30)


832040
Fibonacci without memoization took 0.2251 seconds.
832040
Fibonacci with memoization took 0.0000 seconds.


## Тестирование

In [12]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция wrapper была вызвана 1 раз(а)
Вызов функции wrapper с аргументами ('whole')
word_count took 1.9438 seconds to execute.
Cлово whole встречается 176 раз
